In [1]:
import main

In [2]:
import numpy as np

In [3]:
precursor_1 = main.precursor_FDM(delay_group_num=1)
precursor_1.set_beta([0.0064])
precursor_1.set_lambda([0.08])

In [4]:
material_1 = main.material_FDM(mat_id=0)
material_1.set_XS('XS_scatter',[[0,0.01],[0,0]])
material_1.set_XS('Diffusion_coef',[1.4,0.4])
material_1.set_XS('XS_absorption',[0.01,0.15])
# material_1.set_XS('XS_absorption',[0.01,0.10392197])
material_1.set_XS('XS_nu_fission',[0.007,0.2])
material_1.set_XS('XS_fission_spectrum',[1.0,0.0])

In [5]:
material_2 = main.material_FDM(mat_id=1)
material_2.set_XS('XS_scatter',[[0,0.01],[0,0]])
material_2.set_XS('Diffusion_coef',[1.4,0.4])
material_2.set_XS('XS_absorption',[0.01,0.15])
# material_2.set_XS('XS_absorption',[0.02,0.15])
material_2.set_XS('XS_nu_fission',[0.007,0.2])
material_2.set_XS('XS_fission_spectrum',[1.0,0.0])

In [6]:
material_3 = main.material_FDM(mat_id=2)
material_3.set_XS('XS_scatter',[[0,0.01],[0,0]])
material_3.set_XS('Diffusion_coef',[1.3,0.5])
material_3.set_XS('XS_absorption',[0.008,0.05])
# material_3.set_XS('XS_absorption',[0.018,0.05])
material_3.set_XS('XS_nu_fission',[0.003,0.06])
material_3.set_XS('XS_fission_spectrum',[1.0,0.0])

In [7]:
geo = main.geometry_FDM(x_block=3,y_block=3)
sl = slice(0, 3, None)
geo.set_block_mat(material_3,sl,sl)
geo.set_block_mat(material_1,1,1)
geo.set_block_mat(material_2,1,0)
geo.set_block_mat(material_2,0,1)

geo.set_block_size(x_size=[24,32,24],y_size=[24,32,24])
geo.set_discretized_num(x_dim=[24,32,24],y_dim=[24,32,24])
# geo.set_discretized_num(x_dim=[48,64,48],y_dim=[48,64,48])
geo.check_blocks()

Material layout:
[[2 1 2]
 [1 0 2]
 [2 2 2]]
Block size:
x:[24, 32, 24]
y:[24, 32, 24]
Discretization number:
x:[24, 32, 24]
y:[24, 32, 24]


In [8]:
case = main.solver_FDM(folder_name='TWIGL_Seed_Blanket_2D_SCM_A4_1ms',group_num=2)
case.add_material([material_1,material_2,material_3])
case.set_geometry(geo,transient_mode=True)
case.set_precursor(precursor_1)
case.set_neutron_velocity([1e7,1e5])

In [9]:
# time_steps = np.array([0,0.005])
time_steps = np.linspace(0.0, 0.5, num=501)
# time_steps = np.array([0])
for time_index,time_step in enumerate(time_steps):
    print('Time={} (s)'.format(time_steps[time_index]))
    if time_index==0:
        # To get initial flux distribution
        case.solve_source_iter_correct(max_iter=50,k_tolerance=1e-5,flux_tolerance=1e-4,initial_k=1.0)
        # case.initial_dynamics(transient_algorithm='Implicit_Euler')
        case.initial_dynamics(time_steps=time_steps,transient_algorithm='SCM',vtk_save=False)
    else:
        # Update material
        if time_step<=0.2:
            m1_a2 = 0.15+(time_step-0.0)*(0.15-0.141285)/(0.0-0.2) # Linear 
        elif time_step>0.2 and time_step<=0.4:
            m1_a2 = 0.15175+(time_step-0.2)*(0.15175-0.1535)/(0.2-0.4)
        else:
            m1_a2 = 0.15
        material_1.set_XS('XS_absorption',[0.01,m1_a2])
        case.update_material(0,material_1,mat_type='real')
        # Solve transient problem with SCM
        time_interval = time_steps[time_index]-time_steps[time_index-1]
        case.solve_transient_SCM_2(time_index,time_interval,max_iter=100,k_tolerance=1e-6,flux_tolerance=1e-4,k_outer_tolerance=1e-7)
        # case.solve_transient_Implicit_Euler(time_index,time_interval,max_iter=200,k_tolerance=1e-5,flux_tolerance=1e-4)

Time=0.0 (s)


D:\Users\85136\anaconda3\lib\site-packages\scipy\sparse\linalg\dsolve\linsolve.py:318: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)


####################################
Timestep 0 calculation begins
Iteration 1: Eigenvalue k = 0.8446669513193322
Iteration 2: Eigenvalue k = 0.8899489163397002
Iteration 3: Eigenvalue k = 0.8994594776166441
Iteration 4: Eigenvalue k = 0.9038026479314235
Iteration 5: Eigenvalue k = 0.9064220494043876
Iteration 6: Eigenvalue k = 0.9081900679307713
Iteration 7: Eigenvalue k = 0.9094479494340285
Iteration 8: Eigenvalue k = 0.9103687896329571
Iteration 9: Eigenvalue k = 0.9110545506517275
Iteration 10: Eigenvalue k = 0.9115708863732314
Iteration 11: Eigenvalue k = 0.9119625156101641
Iteration 12: Eigenvalue k = 0.9122610517725338
Iteration 13: Eigenvalue k = 0.9124894190761463
Iteration 14: Eigenvalue k = 0.9126645386803547
Iteration 15: Eigenvalue k = 0.9127990586739625
Iteration 16: Eigenvalue k = 0.9129025182983143
Iteration 17: Eigenvalue k = 0.9129821584390484
Iteration 18: Eigenvalue k = 0.9130435007625038
Iteration 19: Eigenvalue k = 0.9130907698523009
Iteration 20: Eigenvalue k = 0

Iteration 1: Met the criteria and k = 1.0000003724949844
Outer Iteration 6： kD=1.0000003724949844
Amplitude frequency:1.3385913651347536
Amplitude frequency last:1.3325238345539256
Amplitude frequency new:1.3432961327278439
Total Q: 1.0014334703036212
####################################
Outer iteration 7 begins
Iteration 1: Met the criteria and k = 0.9999999814650028
Outer Iteration 7： kD=0.9999999814650028
Amplitude frequency:1.3432961327278439
Amplitude frequency last:1.3385913651347536
Amplitude frequency new:1.3430731246254433
Total Q: 1.0014333586397388
Reached the stopping criterion
Total Q (convergent): 1.0014333586397388
Time=0.003 (s)
####################################
Timestep 3 calculation begins
Total Q (last): 1.0014333586397388
####################################
Outer iteration 1 begins
Iteration 1: Eigenvalue k = 1.0000486150892343
Iteration 2: Eigenvalue k = 1.00005063801904
Iteration 3: Met the criteria and k = 1.0000510033110832
Outer Iteration 1： kD=1.0000510033

####################################
Outer iteration 13 begins
Iteration 1: Met the criteria and k = 0.9999999779554996
Outer Iteration 13： kD=0.9999999779554996
Amplitude frequency:2.3190434143007455
Amplitude frequency last:2.321482364867986
Amplitude frequency new:2.3188263586467155
Total Q: 1.005132294744175
Reached the stopping criterion
Total Q (convergent): 1.005132294744175
Time=0.005 (s)
####################################
Timestep 5 calculation begins
Total Q (last): 1.005132294744175
####################################
Outer iteration 1 begins
Iteration 1: Eigenvalue k = 1.000048657575094
Iteration 2: Eigenvalue k = 1.0000507095007822
Iteration 3: Met the criteria and k = 1.000051100555265
Outer Iteration 1： kD=1.000051100555265
####################################
Outer iteration 2 begins
Iteration 1: Eigenvalue k = 1.0000326375890918
Iteration 2: Met the criteria and k = 1.0000329485808213
Outer Iteration 2： kD=1.0000329485808213
Amplitude frequency:2.550708994511387
Amp

...

####################################
Outer iteration 3 begins
Iteration 1: Met the criteria and k = 0.9999996861942257
Outer Iteration 3： kD=0.9999996861942257
Amplitude frequency:0.06160833965961776
Amplitude frequency last:0.06782165550324656
Amplitude frequency new:0.050155923827766576
Total Q: 1.1220817786517745
####################################
Outer iteration 4 begins
Iteration 1: Met the criteria and k = 1.0000005706727177
Outer Iteration 4： kD=1.0000005706727177
Amplitude frequency:0.050155923827766576
Amplitude frequency last:0.06160833965961776
Amplitude frequency new:0.05754511568108608
Total Q: 1.1220859242982018
####################################
Outer iteration 5 begins
Iteration 1: Met the criteria and k = 0.9999998699680351
Outer Iteration 5： kD=0.9999998699680351
Amplitude frequency:0.05754511568108608
Amplitude frequency last:0.050155923827766576
Amplitude frequency new:0.056173880181628194
Total Q: 1.122085154976439
####################################
Outer ite

####################################
Outer iteration 6 begins
Iteration 1: Met the criteria and k = 1.000000018826772
Outer Iteration 6： kD=1.000000018826772
Amplitude frequency:0.0390608125264862
Amplitude frequency last:0.04015601375771115
Amplitude frequency new:0.039230657239855325
Total Q: 1.1222976488956165
Reached the stopping criterion
Total Q (convergent): 1.1222976488956165
Time=0.47600000000000003 (s)
####################################
Timestep 476 calculation begins
Total Q (last): 1.1222976488956165
####################################
Outer iteration 1 begins
Iteration 1: Met the criteria and k = 1.0000000005202991
Outer Iteration 1： kD=1.0000000005202991
####################################
Outer iteration 2 begins
Iteration 1: Met the criteria and k = 0.9999996955540408
Outer Iteration 2： kD=0.9999996955540408
Amplitude frequency:0.04315372296384086
Amplitude frequency last:0.039230657239855325
Amplitude frequency new:0.03923735033345197
Total Q: 1.1223416819896146
##

Iteration 1: Met the criteria and k = 0.9999999341052529
Outer Iteration 5： kD=0.9999999341052529
Amplitude frequency:0.025714181343869817
Amplitude frequency last:0.021377534089300602
Amplitude frequency new:0.025008383797837333
Total Q: 1.1224749333511455
Reached the stopping criterion
Total Q (convergent): 1.1224749333511455
Time=0.481 (s)
####################################
Timestep 481 calculation begins
Total Q (last): 1.1224749333511455
####################################
Outer iteration 1 begins
Iteration 1: Met the criteria and k = 1.0000000132386997
Outer Iteration 1： kD=1.0000000132386997
####################################
Outer iteration 2 begins
Iteration 1: Met the criteria and k = 0.9999998179703115
Outer Iteration 2： kD=0.9999998179703115
Amplitude frequency:0.027509222177621067
Amplitude frequency last:0.025008383797837333
Amplitude frequency new:0.02517793427442644
Total Q: 1.1225031001465577
####################################
Outer iteration 3 begins
Iteration 

####################################
Outer iteration 1 begins
Iteration 1: Met the criteria and k = 1.0000001129506233
Outer Iteration 1： kD=1.0000001129506233
####################################
Outer iteration 2 begins
Iteration 1: Met the criteria and k = 0.9999999924088511
Outer Iteration 2： kD=0.9999999924088511
Amplitude frequency:0.015180102893429694
Amplitude frequency last:0.013800093539481539
Amplitude frequency new:0.015093196451462931
Total Q: 1.1226190089451187
Reached the stopping criterion
Total Q (convergent): 1.1226190089451187
Time=0.488 (s)
####################################
Timestep 488 calculation begins
Total Q (last): 1.1226190089451187
####################################
Outer iteration 1 begins
Iteration 1: Met the criteria and k = 0.9999999017310647
Outer Iteration 1： kD=0.9999999017310647
####################################
Outer iteration 2 begins
Iteration 1: Met the criteria and k = 0.9999997848074383
Outer Iteration 2： kD=0.9999997848074383
Amplitude

####################################
Outer iteration 1 begins
Iteration 1: Met the criteria and k = 1.0000000174768662
Outer Iteration 1： kD=1.0000000174768662
####################################
Outer iteration 2 begins
Iteration 1: Met the criteria and k = 0.9999999672869477
Outer Iteration 2： kD=0.9999999672869477
Amplitude frequency:0.0063850650110961835
Amplitude frequency last:0.0058046045555419845
Amplitude frequency new:0.006006729404855599
Total Q: 1.1227238023897657
Reached the stopping criterion
Total Q (convergent): 1.1227238023897657
Time=0.498 (s)
####################################
Timestep 498 calculation begins
Total Q (last): 1.1227238023897657
####################################
Outer iteration 1 begins
Iteration 1: Met the criteria and k = 0.9999999616712203
Outer Iteration 1： kD=0.9999999616712203
####################################
Outer iteration 2 begins
Iteration 1: Met the criteria and k = 0.9999999134460837
Outer Iteration 2： kD=0.9999999134460837
Amplitu